In [1]:
import numpy as np
import os
import scipy
import pandas as pd
from scipy.io import savemat,loadmat
from utils.utils import *

# Load raw data from dataset

### ern

In [9]:
import pandas as pd
import numpy as np

def load_ern_data_from_csv(path, t=1.3):
    df = pd.read_csv(path)
    df = df.drop(columns=['Time', 'EOG'])
    INIT = True
    for c in df.columns:
        data = np.array(df[c])
        data = data[np.newaxis,:]
        if INIT:
            data_array = data
            INIT = False
        else:
            data_array = np.concatenate((data_array, data), axis=0)
    # print(len(df.columns), data_array.shape)

    freq = 200
    sample_counts = int(freq*t)
    NOT_CONC = True
    index = np.where(data_array[-1] == 1)[0].tolist()
    for i in index:
        a = data_array[np.newaxis, :-1, i:i+sample_counts]
        if NOT_CONC:
            eeg_data = a
            NOT_CONC = False
        else:
            eeg_data = np.concatenate((eeg_data, a), axis=0)
    return eeg_data
    
def get_ern_dataset(base_path, sub):
    NOT_CONC = True
    for sess in range(1,6):
        if sub < 10:
            file_path = os.path.join(base_path,'Data_S0'+str(sub)+'_Sess0'+str(sess)+'.csv')
        else:
            file_path = os.path.join(base_path,'Data_S'+str(sub)+'_Sess0'+str(sess)+'.csv')
        data = load_ern_data_from_csv(file_path)
        if NOT_CONC:
            eeg_data = data
            NOT_CONC = False
        else:
            eeg_data = np.concatenate((eeg_data, data), axis=0)
    return eeg_data


In [10]:
sub_list = [2, 6, 7, 11, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 26]
base_path = '/home/lyh2020/dataset/ern/train'
NOT_CONC = True
for sub in sub_list:
    data = get_ern_dataset(base_path, sub)
    data = data[np.newaxis]
    if NOT_CONC:
        eeg_data = data
        NOT_CONC = False
    else:
        eeg_data = np.concatenate((eeg_data, data), axis=0)
    print(eeg_data.shape)

(1, 340, 56, 260)
(2, 340, 56, 260)
(3, 340, 56, 260)
(4, 340, 56, 260)
(5, 340, 56, 260)
(6, 340, 56, 260)
(7, 340, 56, 260)
(8, 340, 56, 260)
(9, 340, 56, 260)
(10, 340, 56, 260)
(11, 340, 56, 260)
(12, 340, 56, 260)
(13, 340, 56, 260)
(14, 340, 56, 260)
(15, 340, 56, 260)
(16, 340, 56, 260)


In [11]:
df = pd.read_csv('/home/lyh2020/dataset/ern/train/TrainLabels.csv')
train_label_all = np.array(df['Prediction']).reshape((16,-1))
print(train_label_all.shape)

(16, 340)


In [12]:
savemat(os.path.join(os.path.abspath(''), 'data/ern/sall_raw.mat'),{'data':eeg_data, 'label':train_label_all,})

In [3]:
data_model = loadmat(os.path.join(os.path.abspath(''), 'data/ern/sall_raw.mat'))
data = data_model['data']
data = np.expand_dims(data, 2)
label = data_model['label']
print(data.shape, label.shape)
print(data.min(), data.max())
filter_data = bp_filter(data, fp1=1., fp2=40., f_sample=200)
print(filter_data.min(),filter_data.max())
# normalize_data = scipy.stats.zscore(filter_data, axis=-1)
# print(normalize_data.min(),normalize_data.max())
# savemat(os.path.join(os.path.abspath(''), 'data/ern/sall_bp_z.mat'),{'data':normalize_data, 'label':train_label_all,})

(16, 340, 1, 56, 260) (16, 340)
-4008.509312 2103.329844
-783.6079 716.24744


In [9]:
data_ = filter_data[0]
label_ = label[0]
data_ = np.mean(data_,axis=0)[0]
data_ = np.mean(data_, axis=-1)

Z_centered = filter_data[0]
for i in range(56):
    Z_centered[:,:,i,:] = Z_centered[:,:,i,:] - data_[i]

Z = Z_centered
for i in range(Z.shape[0]):
    Z[i,0] = Z_centered[i,0]/np.sqrt((np.trace(np.dot(Z_centered[i,0], Z_centered[i,0].T))))



(340, 1, 56, 260)


### p300

In [ ]:
sub_list = [1, 2, 3, 4, 6, 7, 8, 9]
for sub in sub_list:
    NOT_CONC = True
    for sees in range(4):
        path = '/home/lyh2020/code/EEG_IB/data/p300/s'+str(sub)+'-'+str(sees+1)+'.mat'
        data_model = loadmat(path)
        for i in range(data_model['runs'][0].shape[0]):
            data = data_model['runs'][0][i]['x'][0][0]
            label = data_model['runs'][0][i]['y'][0][0].flatten()
            for y in range(label.shape[0]):
                if label[y]==-1:
                    label[y]=0
            data = data.transpose(2,0,1)
            if NOT_CONC:
                eeg_data = data
                eeg_label = label
                NOT_CONC = False
            else:
                eeg_data = np.concatenate((eeg_data, data))
                eeg_label = np.concatenate((eeg_label, label))
    savemat(os.path.join(os.path.abspath(''), 'data/p300/s'+str(sub)+'_raw.mat'),{'data':eeg_data, 'label':eeg_label,})
    print(eeg_data.shape, eeg_label.shape)

In [24]:
sub_list = [1, 2, 3, 4, 6, 7, 8, 9]
for sub in sub_list:
    path = '/home/lyh2020/code/EEG_IB/data/p300/s'+str(sub)+'_bp_z.mat'
    data_model = loadmat(path)
    data = data_model['data']
    print(data.min(), data.max())

-4.207197 4.2564096
-4.29014 4.2677517
-4.640484 4.931028
-4.44677 4.6609187
-4.6180396 4.153045
-4.425864 4.2360125
-4.2992034 4.499987
-4.352146 4.352146


In [23]:
sub_list = [1, 2, 3, 4, 6, 7, 8, 9]
for sub in sub_list:
    path = '/home/lyh2020/code/EEG_IB/data/p300/s'+str(sub)+'_raw.mat'
    data_model = loadmat(path)
    data = data_model['data']
    data = np.expand_dims(data, 1)
    label = data_model['label'].flatten()
    print(data.shape, label.shape)
    print(data.min(), data.max())
    filter_data = bp_filter(data, fp1=1., fp2=40., f_sample=256)
    print(filter_data.min(),filter_data.max())
    # normalize_data = (filter_data - np.mean(filter_data))/10
    normalize_data = scipy.stats.zscore(filter_data, axis=-1)
    print(normalize_data.min(),normalize_data.max())

    # savemat(os.path.join(os.path.abspath(''), 'data/p300/s'+str(sub)+'_bp_z.mat'),{'data':normalize_data, 'label':label,})

(3342, 1, 32, 256) (3342,)
-18380.374962832386 18382.076486029393
-17598.615 18943.805
-4.207197 4.2564096
(3324, 1, 32, 256) (3324,)
-9392.531297157882 9391.08686352511
-9054.238 9745.487
-4.29014 4.2677517
(3342, 1, 32, 256) (3342,)
-16204.299667916475 16211.562731298127
-15465.421 16753.367
-4.640484 4.931028
(3318, 1, 32, 256) (3318,)
-9808.597442583117 9810.408023982407
-9439.372 10056.405
-4.44677 4.6609187
(3300, 1, 32, 256) (3300,)
-6820.319520967013 6819.099926234702
-6563.734 6717.004
-4.6180396 4.153045
(3372, 1, 32, 256) (3372,)
-19314.973367976756 19317.210616218195
-19764.973 18476.375
-4.425864 4.2360125
(3336, 1, 32, 256) (3336,)
-12834.670616656866 12832.727477439405
-12285.519 13775.874
-4.2992034 4.499987
(3306, 1, 32, 256) (3306,)
-28586.123539598928 28569.7195040357
-29665.344 27542.115
-4.352146 4.352146


In [ ]:
from datetime import datetime

def get_pos(index):
    a = ''
    add = ['-','-',' ',':',':']
    for i in range(events[index].shape[0]):
        if i == 5:
            a = a + str(round(events[index][i],5))
        else:
            a = a + str(int(events[index][i]))+add[i]
    print(a)
    a = datetime.strptime(str(a),"%Y-%m-%d %H:%M:%S.%f")
    return a
for i in range(131):
    start = get_pos(i)
    end = get_pos(i+1)
    print(end-start, i)

2006-5-19 14:29:9.808
2006-5-19 14:29:10.208
0:00:00.400000 0
2006-5-19 14:29:10.208
2006-5-19 14:29:10.609
0:00:00.401000 1
2006-5-19 14:29:10.609
2006-5-19 14:29:11.01
0:00:00.401000 2
2006-5-19 14:29:11.01
2006-5-19 14:29:11.41
0:00:00.400000 3
2006-5-19 14:29:11.41
2006-5-19 14:29:11.811
0:00:00.401000 4
2006-5-19 14:29:11.811
2006-5-19 14:29:12.211
0:00:00.400000 5
2006-5-19 14:29:12.211
2006-5-19 14:29:12.612
0:00:00.401000 6
2006-5-19 14:29:12.612
2006-5-19 14:29:13.013
0:00:00.401000 7
2006-5-19 14:29:13.013
2006-5-19 14:29:13.413
0:00:00.400000 8
2006-5-19 14:29:13.413
2006-5-19 14:29:13.814
0:00:00.401000 9
2006-5-19 14:29:13.814
2006-5-19 14:29:14.214
0:00:00.400000 10
2006-5-19 14:29:14.214
2006-5-19 14:29:14.615
0:00:00.401000 11
2006-5-19 14:29:14.615
2006-5-19 14:29:15.015
0:00:00.400000 12
2006-5-19 14:29:15.015
2006-5-19 14:29:15.416
0:00:00.401000 13
2006-5-19 14:29:15.416
2006-5-19 14:29:15.817
0:00:00.401000 14
2006-5-19 14:29:15.817
2006-5-19 14:29:16.217
0:00:00.4

In [ ]:
data_model = loadmat('/home/lyh2020/dataset/p300/subject1/session1/eeg_200605191430_epochs.mat')
data = data_model['data']
label = data_model['target']
events = data_model['events']
stimuli = data_model['stimuli'][0]
print(events.shape)
print(stimuli.shape)
print(data.shape)
print(events)

(132, 6)
(132,)
(34, 120422)
[[2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 9.8080e+00]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.0208e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.0609e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.1010e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.1410e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.1811e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.2211e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.2612e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.3013e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.3413e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.3814e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.4214e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.4615e+01]
 [2.0060e+03 5.0000e+00 1.9000e+01 1.4000e+01 2.9000e+01 1.5015e+01]
 [2.0

### bci42a

In [ ]:
def load_single_subject_data(data_path, subject=1, _type='T', bp_filter=False, ems=False, minmax=False, reshape=False):
    raw = mne.io.read_raw_gdf(os.path.join(data_path, 'A0'+str(subject)+_type+'.gdf'))
    raw.info['bads'].extend(['EOG-left', 'EOG-central', 'EOG-right'])

    offset = 2
    tmin, tmax = offset, offset+4-0.004

    events, event_dict = mne.events_from_annotations(raw)
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                    exclude='bads')
    epochs = mne.Epochs(raw, events, event_dict, tmin, tmax, event_repeated = 'merge', proj=True, picks=picks,
                    baseline=None, preload=True)
    if bp_filter: epochs = epochs.filter(4., 40., fir_design='firwin', skip_by_annotation='edge')
    data = epochs['768'].get_data()
    # labels = epochs['769', '770', '771', '772'].events[:, -1]
    labels = loadmat(os.path.join(data_path, 'A0'+str(subject)+_type+'.mat'))["classlabel"].squeeze()
    if ems:
        for trail in range(data.shape[0]):
            data[trail] = exponential_moving_standardize(data[trail])
    if minmax:
        data = min_max(data)
    if reshape:
        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        trail, kernels, chans, samples = data.shape[0], 1, data.shape[-2], data.shape[-1]
        data = data.reshape(trail, kernels, chans, samples).astype('float32')

    labels = labels - 1
    return data, labels   

def load_bci_competition_iv2a_data(data_path, subjects, bp_filter=False, ems=False, minmax=False, reshape=False):
    if subjects == -1:
        subjects = [1,2,3,4,5,6,7,8,9]
    NOT_CONCAT = True
    for i in subjects:
        data_t,labels_t = load_single_subject_data(data_path, i, _type='T', bp_filter=bp_filter, ems=ems, minmax=minmax, reshape=reshape)
        data_e,labels_e = load_single_subject_data(data_path, i, _type='E', bp_filter=bp_filter, ems=ems, minmax=minmax, reshape=reshape)
        data = np.vstack((data_t,data_e))
        labels = np.hstack((labels_t,labels_e))
        data = data[np.newaxis, :]
        labels = labels[np.newaxis, :]
        if NOT_CONCAT:
            eeg_data = data
            eeg_labels = labels
            NOT_CONCAT = False
        else:
            eeg_data = np.vstack((eeg_data,data))
            eeg_labels = np.vstack((eeg_labels,labels))      
    return eeg_data, eeg_labels

In [ ]:
def load_data(base_path, dataset_type, sesson=1):
    path = os.path.join(base_path, 'data', dataset_type, 'sall_bp_ems_'+dataset_type+'.mat')
    dataset = loadmat(path)
    data = dataset['data'][:,:288]
    label = dataset['label'][:,:288]

    return data, label

In [ ]:
dataset = loadmat(os.path.join(os.path.abspath(''), 'data/bci42a/sall_bp_ems_bci42a.mat'))
X_raw = dataset['data'][:,:288]
y_raw = dataset['label'][:,:288]
# for i in range(X_raw.shape[0]):
#     X_raw[i]=min_max(X_raw[i])
print(X_raw.shape, y_raw.shape)

(9, 288, 1, 22, 1000) (9, 288)


In [ ]:
if not os.path.exists(os.path.join(os.path.abspath(''), 'data/bci42a/sall_bp_ems_bci42a.mat')):
    data_path = '/home/lyh2020/dataset/bci42a'
    X_raw, y_raw = load_bci_competition_iv2a_data(data_path, -1, bp_filter=True, minmax=True, reshape=True)
    savemat(os.path.join(os.path.abspath(''), 'data/bci42a/sall_bp_ems_bci42a.mat'),{'data':X_raw,'label':y_raw})
else:
    dataset = loadmat(os.path.join(os.path.abspath(''), 'data/bci42a/sall_bp_ems_bci42a.mat'))
    X_raw = dataset['data'][:,:288]
    y_raw = dataset['label'][:,:288]

print(X_raw.shape, y_raw.shape)

Extracting EDF parameters from /home/lyh2020/dataset/bci42a/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Not setting metadata
Multiple event values for single event times found. Creating new event value to reflect simultaneous events.
Not setting metadata
585 matching events found
No baseline correction applied
0 projection items activated
Loading data for 585 events and 1000 original time points ...


/home/lyh2020/anaconda3/envs/lyh/lib/python3.8/site-packages/mne/io/edf/edf.py:1121: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/home/lyh2020/anaconda3/envs/lyh/lib/python3.8/contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


1 bad epochs dropped
Setting up band-pass filter from 4 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 sec)



KeyboardInterrupt: 